In [62]:
import re
from datetime import datetime
import pandas as pd
import numpy as np
from tkinter import filedialog
from tkinter import *
import os, sys
import glob
from matplotlib.backends.backend_pdf import PdfPages
import time

In [63]:
listOfFiles = []
tempDF = pd.DataFrame()

def get_filenames():
    Tk().withdraw()
    print("Initializing Dialogue... \\nPlease select a file.")
    tk_filenames = filedialog.askdirectory()
    tempDir = tk_filenames
    return tempDir

choosen_dir = get_filenames()

os.chdir(choosen_dir)
for file in glob.glob("*.csv"):
    listOfFiles.append(file)


for file in listOfFiles:
    Table = pd.read_csv(file,sep=',')
    DF1 = pd.DataFrame(Table)
    tempDF = pd.concat([DF1,tempDF],ignore_index=True)

DT = tempDF

Initializing Dialogue... \nPlease select a file.


In [64]:
# Wprowadzamy przedział czasowy z jakiego będą pobieranie dane
#date1 = input('Date range from YYYY-MM-DD') 
#date2 = input('Date range to YYYY-MM-DD')

In [65]:
# W DF zostawaimy tylko dane z wcześniej zadeklarowanego przedziału czasowego
#DT = DT[(DT['KDOC'] >= date1) & (DT['KDOC'] <= date2)]

In [66]:
DT.KY3 = DT.KY3.replace(',', '.',regex = True)
DT.KY5 = DT.KY5.replace(',', '.',regex = True)
DT.OKO = DT.OKO.replace(',', '.',regex = True)
DT.MIESO_PROC = DT.MIESO_PROC.replace(',', '.',regex = True)
DT['KY3'] = DT['KY3'].astype(float)
DT['KY5'] = DT['KY5'].astype(float)
DT['OKO'] = DT['OKO'].astype(float)
DT['MIESO_PROC'] = DT['MIESO_PROC'].astype(float)

In [67]:
DT = DT[['KE','KS','KW','KSE','KP','KDOC','KH','KRO','KOJ','KRM','KMT','KNR','KDUR','KY1','KY2','KY3','KY4','KY5','OKO','T_PRZYR','MIESO_PROC','T_INDEX']]
knur = DT['KP'] == 2
DT = DT[knur]

In [68]:
DT['KDOC'] = pd.to_datetime(DT['KDOC'])
DT['KDUR'] = pd.to_datetime(DT['KDUR'])
DT['DNI'] = 0
DT['DNI'] = DT['KDOC'] - DT['KDUR']
DT['DNI'] = DT['DNI'] / np.timedelta64(1, 'D')

In [69]:
DT = DT[DT['KDOC'].dt.year == 2020]
DT = DT.drop_duplicates()

In [70]:
DT['KRO'] = DT['KRO'].apply(str)
DT['KRM'] = DT['KRM'].apply(str)
DT['combined'] = DT['KRO']+"|"+DT['KRM']
clear_dict = {}
clear_dict['10|10'] = 1
clear_dict['20|20'] = 2
clear_dict['60|60'] = 3
clear_dict['70|70'] = 4
clear_dict['80|80'] = 5
clear_dict['90|90'] = 6
clear_dict['30|30'] = 7
clear_dict['50|50'] = 8
clear_dict['40|40'] = 9
DT['cleared'] = DT['combined'].apply(lambda y:clear_dict[y] if y in clear_dict.keys() else y)
rest = DT[DT['cleared'].str.contains('|') == True]

In [71]:
wbp = DT.where(DT['cleared'] == 1).dropna()
pbz = DT.where(DT['cleared'] == 2).dropna()
hamp = DT.where(DT['cleared'] == 3).dropna()
dur = DT.where(DT['cleared'] == 4).dropna()
piet = DT.where(DT['cleared'] == 5).dropna()
l990 = DT.where(DT['cleared'] == 6).dropna()
zb = DT.where(DT['cleared'] == 7).dropna()
zps = DT.where(DT['cleared'] == 8).dropna()
pula = DT.where(DT['cleared'] == 9).dropna()

In [72]:
def createDT(name,DT):
    if DT['KP'].all() == 1:
        temp = pd.DataFrame(columns=['Rasa',"Liczba Knurów",'Wiek w dniu oceny','Masa ciała w dniu oceny (kg)','Średnia grubość słoniny standar. (mm)',
                    'Wysokość "oka" polędwicy standar. (mm)','Przyrost dzienny standar. (g)', 'Procentowa zawartość mięsa standar.',
                    'Indeks (pkt)'],index =[1])
        temp['Rasa'] = name
        return temp
    else:
        temp = pd.DataFrame(columns=['Rasa',"Liczba Loch",'Wiek w dniu oceny','Masa ciała w dniu oceny (kg)','Średnia grubość słoniny standar. (mm)',
                    'Wysokość "oka" polędwicy standar. (mm)','Przyrost dzienny standar. (g)', 'Procentowa zawartość mięsa standar.',
                    'Indeks (pkt)'],index =[1])
        temp['Rasa'] = name
        return temp

In [73]:
def new_count(DT,DT2):
    DT2['Liczba Knurów'] = len(DT)
    DT2['Wiek w dniu oceny'] = DT['DNI'].mean()
    DT2['Wiek w dniu oceny'] = DT2['Wiek w dniu oceny'].round(decimals=0)
    DT2['Wiek w dniu oceny'] = DT2['Wiek w dniu oceny'].astype(int)
    DT2['Masa ciała w dniu oceny (kg)'] = np.around(DT['KY1'].mean())
    DT2['Masa ciała w dniu oceny (kg)'] = DT2['Masa ciała w dniu oceny (kg)'].astype(int)
    DT2['Średnia grubość słoniny standar. (mm)'] = np.around(((DT['KY3']+DT['KY5'])/2).mean(),decimals=2)
    DT2['Wysokość "oka" polędwicy standar. (mm)'] = np.around(DT['OKO'].mean(),decimals=2)
    DT2['Przyrost dzienny standar. (g)'] = np.around(DT['T_PRZYR'].mean())
    DT2['Przyrost dzienny standar. (g)'] = DT2['Przyrost dzienny standar. (g)'].astype(int)
    DT2['Procentowa zawartość mięsa standar.'] = np.around(DT['MIESO_PROC'].mean(),decimals=2)
    DT2['Indeks (pkt)'] = np.around(DT['T_INDEX'].mean())
    DT2['Indeks (pkt)'] = DT2['Indeks (pkt)'].astype(int)
    return DT2

In [76]:
WBP = createDT('Wbp',wbp)
new_count(wbp,WBP)

PBZ = createDT('Pbz',pbz)
new_count(pbz,PBZ)

#HAMP = createDT('Hampshire',hamp)
#new_count(hamp,HAMP)

DUR = createDT('Duroc',dur)
new_count(dur,DUR)

PIET = createDT('Pietrain',piet)
new_count(piet,PIET)

L990 = createDT('Linia 990',l990)
new_count(l990,L990)

ZB = createDT('Złotnicka biała',zb)
new_count(zb,ZB)

ZPS = createDT('Złotnica pstra',zps)
new_count(zps,ZPS)

PULA = createDT('Puławska',pula)
new_count(pula,PULA)

REST = createDT('Mieszańce',rest)
new_count(rest,REST)

#table to display
last = pd.concat([WBP,PBZ,DUR,PIET,L990,ZB,ZPS,PULA,REST])

In [82]:
last

,Rasa,Liczba Knurów,Wiek w dniu oceny,Masa ciała w dniu oceny (kg),Średnia grubość słoniny standar. (mm),"Wysokość ""oka"" polędwicy standar. (mm)",Przyrost dzienny standar. (g),Procentowa zawartość mięsa standar.,Indeks (pkt)
1,Wbp,3271,172,110,10.21,58.96,659,59.42,119
1,Pbz,5856,174,114,10.08,58.02,675,59.37,121
1,Duroc,406,176,116,10.05,61.37,677,60.68,114
1,Pietrain,291,186,121,8.14,65.28,648,63.59,124
1,Linia 990,548,169,104,9.80,58.72,636,59.39,103
1,Złotnicka biała,169,196,101,18.31,46.33,514,50.48,84
1,Złotnica pstra,87,205,88,19.65,40.91,425,47.94,60
1,Puławska,3081,186,106,12.75,52.93,565,56.14,94
1,Mieszańce,3529,172,113,10.24,57.96,675,59.02,120


In [77]:
def createDTCity():
    temp = pd.DataFrame(columns=['Filia "POLSUS"',"Liczba Knurów",'Wiek w dniu oceny','Masa ciała w dniu oceny (kg)','Średnia grubość słoniny standar. (mm)',
                     'Wysokość "oka" polędwicy standar. (mm)','Przyrost dzienny standar. (g)', 'Procentowa zawartość mięsa standar.',
                     'Indeks (pkt)'])
    temp['Filia "POLSUS"'] = ["Wrocław", "Bydgoszcz","Lublin","Zielona Góra","Łódź","Kraków","Warszawa","Opole","Rzeszów","Białystok","Gdańsk",
                     "Katowice","Kielce","Olsztyn","Poznań","Koszalin"]
    temp = pd.DataFrame(temp).set_index('Filia "POLSUS"')
    return temp

In [78]:
dic_miasta = {2:'Wrocław',
             4:'Bydgoszcz',
             6:'Lublin',
             8:'Zielona Góra',
             10:'Łódź',
             12:'Kraków',
             14:'Warszawa',
             16:'Opole',
             18:'Rzeszów',
             20:'Białystok',
             22:'Gdańsk',
             24:'Katowice',
             26:'Kielce',
             28:'Olsztyn',
             30:'Poznań',
             32:'Koszalin'}

In [79]:
def round_float(s):
    '''1. if s is float, round it to 0 decimals
       2. else return s as is
    '''
    import re
    m = re.match("(\d+\.\d+)",s.__str__())
    try:
        r = round(float(m.groups(0)[0]),2)
    except:
        r = s
    return r

def split_it(days):
    return re.compile('^\d{3}', days)

def split_2(days2):
    return re.findall('(\[)', days2)

def city_count(DT,PDT):
    for i, sub in PDT.groupby('KW'):
        DT.loc[dic_miasta[i]]['Liczba Knurów'] = sub.KY1.agg(len)
        DT.loc[dic_miasta[i]]['Masa ciała w dniu oceny (kg)'] = sub.KY1.mean()
        DT.loc[dic_miasta[i]]['Wiek w dniu oceny'] = sub.DNI.mean()
        DT.loc[dic_miasta[i]]['Średnia grubość słoniny standar. (mm)'] = ((sub.KY3 + sub.KY5)/2).mean()
        DT.loc[dic_miasta[i]]['Wysokość "oka" polędwicy standar. (mm)'] = sub.OKO.mean()
        DT.loc[dic_miasta[i]]['Przyrost dzienny standar. (g)'] = sub.T_PRZYR.mean()
        DT.loc[dic_miasta[i]]['Procentowa zawartość mięsa standar.'] = sub.MIESO_PROC.mean()
        DT.loc[dic_miasta[i]]['Indeks (pkt)'] = sub.T_INDEX.mean()


def roundup(DT):
    DT.dropna(inplace=True)
    DT['Wiek w dniu oceny'] = DT['Wiek w dniu oceny'].astype(int)
    DT['Masa ciała w dniu oceny (kg)'] = DT['Masa ciała w dniu oceny (kg)'].astype(int)
    DT['Średnia grubość słoniny standar. (mm)'] = DT['Średnia grubość słoniny standar. (mm)'].apply(round_float)
    DT['Wysokość "oka" polędwicy standar. (mm)'] = DT['Wysokość "oka" polędwicy standar. (mm)'].apply(round_float)
    DT['Procentowa zawartość mięsa standar.'] = DT['Procentowa zawartość mięsa standar.'].apply(round_float)
    DT['Przyrost dzienny standar. (g)'] = DT['Przyrost dzienny standar. (g)'].astype(int)
    DT['Indeks (pkt)'] = DT['Indeks (pkt)'].astype(int)
    return DT


In [80]:
city_wbp = createDTCity()
city_count(city_wbp,wbp)
roundup(city_wbp)

city_pbz = createDTCity()
city_count(city_pbz,pbz)
roundup(city_pbz)

city_hamp = createDTCity()
city_count(city_hamp,hamp)
roundup(city_hamp)

city_dur = createDTCity()
city_count(city_dur,dur)
roundup(city_dur)

city_piet = createDTCity()
city_count(city_piet,piet)
roundup(city_piet)

city_l990 = createDTCity()
city_count(city_l990,l990)
roundup(city_l990)

city_zb = createDTCity()
city_count(city_zb,zb)
roundup(city_zb)

city_zps = createDTCity()
city_count(city_zps,zps)
roundup(city_zps)

city_pula = createDTCity()
city_count(city_pula,pula)
roundup(city_pula)

city_rest = createDTCity()
city_count(city_rest,rest)
roundup(city_rest)

#table to display
last2 = pd.concat([city_wbp,city_pbz,city_hamp,city_dur,city_piet,city_l990,city_zb,city_zps,city_pula,city_rest],keys=['Wbp','Pbz','Hampshire','Duroc','Pietrain','Linia 990','Złotnicka biała','Złotnicka pstra','Puławska','Mieszańce'])

In [81]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

last2

Liczba Knurów  Wiek w dniu oceny  \
                Filia "POLSUS"                                    
Wbp             Wrocław                  256                180   
                Bydgoszcz                536                175   
                Lublin                    74                178   
                Łódź                     158                166   
                Warszawa                  34                190   
                Opole                     50                181   
                Rzeszów                   50                176   
                Gdańsk                   213                179   
                Olsztyn                  394                170   
                Poznań                  1506                169   
Pbz             Wrocław                   86                182   
                Bydgoszcz                883                176   
                Lublin                    34                188   
                Łódź                     394                163   
                Kraków                   106                168   
                Warszawa                 266                181   
                Opole                    294                178   
                Rzeszów                   40                170   
                Białystok                 14                176   
                Gdańsk                   648                183   
                Katowice                 349                174   
                Kielce                   219                163   
                Olsztyn                  270                167   
                Poznań                  2253                171   
Duroc           Bydgoszcz                 90                178   
                Łódź                      29                168   
                Warszawa                  11                183   
                Opole                     17                195   
                Gdańsk                     4                209   
                Olsztyn                   81                174   
                Poznań                   174                172   
Pietrain        Bydgoszcz                 62                190   
                Łódź                       8                195   
                Opole                     52                205   
                Gdańsk                     6                199   
                Katowice                  12                172   
                Olsztyn                   54                185   
                Poznań                    97                173   
Linia 990       Poznań                   548                169   
Złotnicka biała Zielona Góra              11                207   
                Kraków                    13                203   
                Poznań                   145                194   
Złotnicka pstra Wrocław                    7                221   
                Gdańsk                    38                208   
                Poznań                    42                199   
Puławska        Bydgoszcz                732                181   
                Lublin                   817                188   
                Kraków                    23                198   
                Warszawa                 964                185   
                Opole                     57                185   
                Białystok                 73                182   
                Gdańsk                   305                193   
                Kielce                    16                162   
                Olsztyn                   50                170   
                Koszalin                  44                195   
Mieszańce       Wrocław                  125                179   
                Bydgoszcz                668                171   
                Lublin                   286                180   
           